In [29]:
from modules.kaggle_dataloader import load_data

boards_train, labels_train, boards_test, labels_test = load_data("../hex_games_1_000_000_size_7.csv", 0.9, 10000)
dimensions = 7

Number of boards: 10000
First board: [-1  1  0  0 -1 -1  1  0  1  1 -1 -1  1  1  0  1  1 -1  1  1  0  1  1 -1
  1  0 -1  0 -1  0  1 -1  1  0 -1 -1  1  0 -1  1 -1 -1  1 -1  0  1 -1  0
 -1]
First label: 1
Training samples: 9000
Test samples: 1000
First training board: [-1  1  0  0 -1 -1  1  0  1  1 -1 -1  1  1  0  1  1 -1  1  1  0  1  1 -1
  1  0 -1  0 -1  0  1 -1  1  0 -1 -1  1  0 -1  1 -1 -1  1 -1  0  1 -1  0
 -1]
First training label: 1
First test board: [ 0 -1  1  0  0 -1  1  1  0  1  1  1 -1  1  1  0  1 -1 -1  0  1  1  1  0
  1 -1  0 -1  1 -1 -1 -1 -1  1  0 -1 -1 -1  1  0 -1  1 -1  1 -1 -1  0  0
  0]
First test label: 0


In [30]:
# setting up training graphs

from GraphTsetlinMachine.graphs import Graphs

#symbols for nodes
symbols = ["X", "O", "E"]
for i in range(dimensions):
    symbols.append(f"Row:{i}")
    symbols.append(f"Column:{i}")

#initialize the graphs object
graphs_train = Graphs(
    number_of_graphs=len(boards_train),
    symbols=symbols
)

#e.g. 49 for 7x7 board
number_of_nodes = dimensions * dimensions

#set the number of nodes for each graph
for graph_id in range(len(boards_train)):
    graphs_train.set_number_of_graph_nodes(graph_id, number_of_nodes)

#function to get neighbors of a node (cell)
def get_neighbors(row, column, dimensions):
    neighbors = []
    potential_neighbors = [
        (row - 1, column),  #north-west
        (row + 1, column),  #south-east
        (row, column - 1),  #west
        (row, column + 1),  #east
        (row - 1, column + 1),  #north-east
        (row + 1, column - 1),  #south-west
    ]


    for neighbor_row, neighbor_column in potential_neighbors:
        if 0 <= neighbor_row < dimensions and 0 <= neighbor_column < dimensions:
            neighbors.append((neighbor_row, neighbor_column))

    return neighbors


graphs_train.prepare_node_configuration()

#add the nodes to each graph
for graph_id, board in enumerate(boards_train):
    for node_id in range(number_of_nodes):
        row = node_id // dimensions
        column = node_id % dimensions
        neighbors = get_neighbors(row, column, dimensions)

        graphs_train.add_graph_node(graph_id, node_id, len(neighbors))

graphs_train.prepare_edge_configuration()

#add the edges
for graph_id, board in enumerate(boards_train):
    for node_id in range(number_of_nodes):
        row = node_id // dimensions
        column = node_id % dimensions
        neighbors = get_neighbors(row, column, dimensions)

        for (neighbor_row, neighbor_column) in neighbors:
            neighbor_id = neighbor_row * dimensions + neighbor_column
            graphs_train.add_graph_node_edge(graph_id, node_id, neighbor_id, "adjacent_cell")


#add the node properties
for graph_id, board in enumerate(boards_train):
    for node_id in range(number_of_nodes):
        row = node_id // dimensions
        column = node_id % dimensions

        #X means cell is occupied by player 0, O is player 1, and E is empty
        cell_value = board[node_id]
        if cell_value == 'X':
            graphs_train.add_graph_node_property(graph_id, node_id, "X")
        elif cell_value == 'O':
            graphs_train.add_graph_node_property(graph_id, node_id, "O")
        elif cell_value == 'E':
            graphs_train.add_graph_node_property(graph_id, node_id, "E")

        graphs_train.add_graph_node_property(graph_id, node_id, f"Row:{row}")
        graphs_train.add_graph_node_property(graph_id, node_id, f"Column:{column}")

graphs_train.encode()

In [31]:
#setting up the test graphs
#its the same as the training graphs but we need to set them up separately

graphs_test = Graphs(
    number_of_graphs=len(boards_test),
    init_with=graphs_train
)

for graph_id in range(len(boards_test)):
    graphs_test.set_number_of_graph_nodes(graph_id, number_of_nodes)

graphs_test.prepare_node_configuration()

#add nodes
for graph_id, board in enumerate(boards_test):
    for node_id in range(number_of_nodes):
        row = node_id // dimensions
        column = node_id % dimensions
        neighbors = get_neighbors(row, column, dimensions)

        graphs_test.add_graph_node(graph_id, node_id, len(neighbors))

graphs_test.prepare_edge_configuration()

#add node edges
for graph_id, board in enumerate(boards_test):
    for node_id in range(number_of_nodes):
        row = node_id // dimensions
        column = node_id % dimensions
        neighbors = get_neighbors(row, column, dimensions)

        for (neighbor_row, neighbor_column) in neighbors:
            neighbor_id = neighbor_row * dimensions + neighbor_column
            graphs_test.add_graph_node_edge(graph_id, node_id, neighbor_id, "adjacent_cell")

#add node properties
for graph_id, board in enumerate(boards_test):
    for node_id in range(number_of_nodes):
        row = node_id // dimensions
        column = node_id % dimensions
        cell_value = board[node_id]

        if cell_value == 'X':
            graphs_test.add_graph_node_property(graph_id, node_id, "X")
        elif cell_value == 'O':
            graphs_test.add_graph_node_property(graph_id, node_id, "O")
        elif cell_value == 'E':
            graphs_test.add_graph_node_property(graph_id, node_id, "E")

        graphs_test.add_graph_node_property(graph_id, node_id, f"Row:{row}")
        graphs_test.add_graph_node_property(graph_id, node_id, f"Column:{column}")

graphs_test.encode()

In [32]:
#training logic
from modules.trainer import train

train(
    graphs_train=graphs_train,
    labels_train=labels_train,
    graphs_test=graphs_test,
    labels_test=labels_test,
    dimensions=dimensions,
    number_of_clauses=15000,
    T=750,
    s=1.0,
    depth=1,
    accuracy_threshhold=1,
    epochs=500000
)


Initialization of sparse structure.


FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Training Accuracy',
              'type': 'scatter',
              'uid': '0ace7087-6ed5-49bc-af89-7a5536f6365f',
              'x': [],
              'y': []},
             {'mode': 'lines',
              'name': 'Testing Accuracy',
              'type': 'scatter',
              'uid': '6ec13415-9f2f-4510-8537-81724ac2a3e2',
              'x': [],
              'y': []},
             {'line': {'color': 'gray', 'dash': 'dash'},
              'mode': 'lines',
              'name': 'Test Regression',
              'type': 'scatter',
              'uid': 'f8c4b968-6563-4edb-ac0f-47fcac38c3de',
              'x': [],
              'y': []}],
    'layout': {'annotations': [{'font': {'size': 18},
                                'showarrow': False,
                                'text': 'Train: - , Test: -',
                                'x': 0.5,
                                'xref': 'paper',
                          

KeyboardInterrupt: 